---
title: Hierarchical Clustering using Scikit-Learn
math: 
    '\abs': '\left\lvert #1 \right\rvert' 
    '\norm': '\left\lvert #1 \right\rvert' 
    '\Set': '\left\{ #1 \right\}'
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\M{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
---

**CS5483 Data Warehousing and Data Mining**
___

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
from sklearn import datasets, preprocessing
from sklearn.cluster import AgglomerativeClustering
from sklearn.pipeline import make_pipeline
from util import plot_dendrogram

%matplotlib widget

In this notebook, we continue to cluster the instances in the iris 2D dataset.

## Agglomerative clustering

We first import the iris dataset from `sklearn.datasets` and store it as a DataFrame.

In [ ]:
# load the dataset from sklearn
dataset = datasets.load_iris()

# create a DataFrame to help further analysis
df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
df["target"] = dataset.target
df.target = df.target.astype("category")
df.target = df.target.cat.rename_categories(dataset.target_names)
df  # display an overview of the data

To normalize the features followed by [agglomerative clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html), we create a pipeline as follows:

```python
from sklearn.cluster import AgglomerativeClustering
```

In [ ]:
agnes_minmax_normalized = make_pipeline(
    preprocessing.MinMaxScaler(),
    AgglomerativeClustering(
        n_clusters=3, linkage="complete", memory="private", compute_distances=True
    ),
)
agnes_minmax_normalized

In the above, we configured the agglomerative clustering algorithm to use complete-linkage to return 3 clusters.
- By setting the parameter `memory`, the solution will be cached to the specified folder `private`.
- By setting compute_distances to `True`, the cluster distances are computer for dendrogram visualization later on.

To cluster based on the two input features `petal length (cm)` and `petal width (cm)`:

In [ ]:
feature1, feature2 = "petal length (cm)", "petal width (cm)"
cluster_labels = agnes_minmax_normalized.fit_predict(df[[feature1, feature2]])

plt.figure(num=1, figsize=(10, 5))
plt.subplot(121)
plt.scatter(df[feature1], df[feature2], c=cluster_labels)
plt.title("Cluster assignment")
plt.xlabel(feature1)
plt.ylabel(feature2)
plt.subplot(122)
plt.scatter(df[feature1], df[feature2], c=dataset["target"])
plt.title("Class (ground truth)")
plt.xlabel(feature1)
plt.show()

The [`fit_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering.fit_predict) method fits and returns the cluster labels for the given data.

::::{exercise}
:label: ex:1
`AgglomerateClustering` does not implement [a separate `predict` method, unlike `KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.predict). Why?
::::

YOUR ANSWER HERE

### Dendrogram

Further details of a clustering solution can be obtained from the fitted properties:[^fp]

[^fp]: `sklearn` uses [trailing underscore](https://github.com/rasbt/python-machine-learning-book/blob/master/faq/underscore-convention.md) to denote fitted properties.

In [ ]:
agnes = agnes_minmax_normalized["agglomerativeclustering"]
print("Cluster labels for each sample:", *agnes.labels_)
print("Children of each non-leaf node:", *agnes.children_)
print("Distances between nodes:", *agnes.distances_.round(3))
print("Number of leaves:", agnes.n_leaves_)
print("Number of connected components:", agnes.n_connected_components_)

To plot the dendrogram, we will use the function [`plot_dendrogram`](util.py) input as follows:

```python
from util import plot_dendrogram
```

The code is adopted from [an example in `sklearn`](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html), which uses [`dendrogram` from `scipy.cluster.hierarchy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html). To generate the dendrogram:

In [ ]:
plt.figure(num=2, figsize=(10, 5))
plot_dendrogram(agnes, distance_sort=True)
plt.title("Dendrogram for complete-linkage method on iris dataset")
plt.ylabel("cophenetic distance")
plt.xlabel("instance index")
plt.show()

:::{tip}
You can improve the visualization using the [additional parameters](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) passed to `scipy.cluster.hierarchy.dendrogram`.
:::

::::{exercise}
:label: ex:2
Complete the following code to assign to `agnes_minmax_normalized` a pipelined agglomerative clusterer that 
- min-max normalizes the features, and
- clusters the instances of features specified by `feature1` and `feature2` into `k` clusters according to the linkage method specified by `linkage`.

Observe that the dendrogram and clustering solution change when you change the linkage, features, and number of clusters.

::::

In [ ]:
@interact(
    linkage=["ward", "complete", "average", "single"],
    feature1=dataset.feature_names,
    feature2=dataset.feature_names,
)
def analyze_agnes(
    linkage,
    feature1=dataset.feature_names[2],
    feature2=dataset.feature_names[3],
    k=widgets.IntSlider(3, 1, 4, continuous_update=False),
):
    # YOUR CODE HERE
    raise NotImplementedError()
    
    agnes = agnes_minmax_normalized["agglomerativeclustering"]
    plt.figure(num=3, figsize=(10, 10))

    plt.subplot(211)
    plot_dendrogram(agnes)
    plt.title(f"Dendrogram for {linkage}-linkage method on iris dataset")
    plt.ylabel("cophenetic distance")
    plt.xlabel("instance index")

    plt.subplot(223)
    plt.scatter(df[feature1], df[feature2], c=agnes.labels_)
    plt.title("Cluster assignment")
    plt.ylabel(feature2)
    plt.xlabel(feature1)

    plt.subplot(224)
    plt.scatter(df[feature1], df[feature2], c=dataset["target"])
    plt.title("Class (ground truth)")
    plt.xlabel(feature1)

    plt.show()